# **2048 Solver**

Author: Frank Looi

Written for: CS152 Final Project

LO Synthesized: *#search*

This Python implementation solves the [2048 puzzle](https://gabrielecirulli.github.io/2048/) using Artificial Intelligence. Essentially, this game is about getting one tile to 2048 by merging identical tiles (eg: 2 + 2 = 4, ..., 1024 + 1024 = 2048) where each move subsequently generates a new random tile. The most common search algorithm used in 2048 solvers are minimax algorithms, where the solver AI and the tile generator each take turn to make moves while the solver AI maximizes its score. 

Instead of minimax, however, this implementation uses the [expectimax search algorithm](https://web.uvic.ca/~maryam/AISpring94/Slides/06_ExpectimaxSearch.pdf) where the solver performs maximization over all possible moves (up, down, left, and right) based on the expected utility of each move. At each decision node, the algorithm uses depth-first search to branch into subtrees for all possible tile spawns (chance nodes) to calculate the expected utility for a given move. 

Using expectimax, however, has a downside of not being able to use pruning methods. Therefore, the algorithm needs to utilize depth-limited search and scoring heuristics to optimize the brute force search.


## Performance of the Solver

### Depth-limited search
Generally, the deeper the search, the more informed the search algorithm can be about the subsequent states. However, this accuracy has a significant effect on speed of the algorithm, given that it takes about 1000 moves on average to complete the puzzle. The depth-limit of this algorithm is most effective between 3 - 6. A depth-limit of 4 generates about 75% win rate at an average time of 2m 40s (6 moves per second), while a depth-limit of 3 generates about 50% win rate at an average time of 16s (60 moves per second).

### Scoring Heuristics
Several heuristics are used to direct the optimization algorithm towards favorable positions. The heuristics are combined to compute a positional score which determines how good a given board position is, while the optimization search aims to maximize the average score of all possible board positions. The precise choice of heuristic has a huge effect on AI gameplay and consequently, the performance of the algorithm. 

**1. Corners**

This heuristic grants a high score for having the largest tile on the top left corner. The idea behind this is to preserve the largest value in a corner position by fixing it in a row that is fully occupied at all times, which is a common strategy for human successful gameplay. This also means that the algorithm minimizes down/right movements (the forbidden move) and favor up/left movements during merging. The highest tile value is multiplied with a weight, and this heuristic score becomes larger as the game progresses, so as to (almost) definitively secure the largest value the corner spot, since one displacement can result in a loss (since the algorithm has to rebuild whatever tile is on the top left corner and may run out of smaller tiles to merge).

**2. Empty Tiles**

The larger the number of empty tiles in the grid, the more move options the solver has. Move options can quickly run out when the game board gets too cramped, for example left/right movements are impossible when the columns are fully filled and horizontal merging is not possible. This heuristic gives a bonus for having as many empty tiles as possible at any point during solving.

**3. Monotonicity**

This heuristic works with the corner heuristic by creating monotonically decreasing tiles in the top row. This facilitates merging to the left and towards the largest tile value. For instance, if the top row is [128, 64, 32, 16], the algorithm can merge left consecutively to build a 256 tile once a merge on the 16 is done. Similar to the corner heuristic, the multiplication of weight by tile value also ensures progressively larger values are stacked in the top row, delaying merging until they could be merged with the corner value altogether. This heuristic also applies monotonicity throughout the board to facilitate merging (working alongside empty tiles), though with smaller weights than the top row.

**4. Smoothness**

Using monotonicity heuristic alone tends to create grid positions where adjacent tiles are decreasing in value but in order to merge, adjacent tiles need to be the same value. Therefore, this heuristic attempts to penalize adjacent tiles with a large value difference (e.g. if 1024 is next to 2) because they cannot readily be merged, also working alongside empty tiles to minimize ineffective tiles. Value differences are calculated in logarithmic space, since the numbers are all exponents of 2.

## Implementation

**Grid Class:** Creates an n-dimensional array with n elements to represent a square grid with size n. Even though this game is usually played on a 4x4 grid, this class is coded to be generalizable to any grid size. This data structure is used to ensure iterability and multi-dimensional indexing. For instance, one can specify the bottom right tile just by using Grid().data[-1][-1]. 

The attributes of this class are:
*   size, which is the grid size
*   data, which is the value of a tile in the grid

The methods of this class include: finding the largest tile, setting values to certain tiles, finding empty cells, etc. that form the backbone of the 2048 Puzzle. 

**Game Class:** The central game manager for the 2048-puzzle, which instantiates a Grid object, runs expectimax and apply its moves on the grid until the goal is accomplished (or if it fails).

The attributes of this class are:
*   grid, a grid object for the 2048 game
*   possibility, the possibility of a random spawn having tile value of 2
*   init_tiles, the number of pre-filled tiles when game starts
*   interactive, whether or not to print the grid state at each decision node and the AI's moves
*   depth, the depth-limit for expectimax search

The methods of this class include: starting the game with specific parameters, determining if game is over, spawning new tiles between of 2 or 4 on empty tiles, etc. 

### **Grid Class**

In [0]:
!pip install colorama
from colorama import Fore, Back
from copy import deepcopy

n = 4   # 4x4 grid
directions = [UP, DOWN, LEFT, RIGHT] = range(4)  # enumerate directions
vectors = (UP_VEC, DOWN_VEC, LEFT_VEC, RIGHT_VEC) = ((-1,0),(1,0),(0,-1),(0,1))

class Grid:
    """Represents a Grid for the 2048-puzzle.
    
    Parameters
    ----------
    size : int
        Creates a square grid of a specific size.
    data : list of lists
        Represents value of each tile in the grid
        
    """
    def __init__(self, size = n):
        self.size = size
        self.data = [[0] * self.size for i in xrange(self.size)]
         
    def clone(self):
        # makes a clone of the grid 
        grid_copy = Grid()
        grid_copy.data = deepcopy(self.data)
        grid_copy.size = self.size
        return grid_copy
      
    def equal(self, other):
        # checks if two grids have the same tile values
        if self.size != other.size:
            return False
        for i in xrange(self.size):
            for j in xrange(self.size):
                if self.data[i][j] != other.data[i][j]:
                    return False
        return True
        
    def set_value(self, pos, value):
        # assign value to a tile in a specified position
        x = pos[0]
        y = pos[1]
        self.data[x][y] = value
        
    def get_empty_tiles(self):
        # find list of tiles with tile value of 0
        tiles = []
        for x in xrange(self.size):
            for y in xrange(self.size):
                if self.data[x][y] == 0:
                    tiles.append((x,y))
        return tiles
      
    def get_largest_tile(self):
        # find tile with largest number
        max_tile = 0
        for x in xrange(self.size):
            for y in xrange(self.size):
                max_tile = max(max_tile, self.data[x][y])
        return max_tile
      
    def can_insert(self, pos):
        # return True if tile is empty
        if self.get_tile_value(pos) == 0:
            return True
        else:
            return False
    
    def move(self, dir):
        # move grid in a direction by calling the appropriate function
        dir = int(dir)
        if dir == UP:
            return self.move_up(True)
        elif dir == DOWN:
            return self.move_up(False)
        elif dir == LEFT:
            return self.move_left(True)
        elif dir == RIGHT:
            return self.move_left(False)
          
    def move_up(self, up):
        # move tiles up
        r = range(self.size) if up else range(self.size -1, -1, -1)
        moved = False
        for row in range(self.size):
            tiles = []
            # for each column, record tile if value is non-zero
            # ascending order if up, and descending if down
            for column in r:
                tile = self.data[column][row]
                if tile != 0:
                    tiles.append(tile)
            self.merge(tiles)
            # assign the recorded value to the appropriate column
            for column in r:
                value = tiles.pop(0) if tiles else 0
                if self.data[column][row] != value:
                    moved = True
                self.data[column][row] = value
        return moved

    def move_left(self, left):
        # move tiles left
        r = range(self.size) if left else range(self.size -1, -1, -1)
        moved = False
        # similar to moving up, but loop column first
        for column in range(self.size):
            tiles = []
            for row in r:
                tile = self.data[column][row]
                if tile != 0:
                    tiles.append(tile)
            self.merge(tiles)
            for row in r:
                value = tiles.pop(0) if tiles else 0
                if self.data[column][row] != value:
                    moved = True
                self.data[column][row] = value
        return moved
      
    def merge(self,tiles):
        # merge two equal tiles after each move
        if len(tiles) <= 1:
            # no merging necessary 
            return tiles
        i = 0
        while i < len(tiles) - 1:
            # value in the tile doubles if merged with equal value
            if tiles[i] == tiles[i+1]:
                tiles[i] *= 2
                del tiles[i+1]
            i += 1
      
    def can_move(self, directions = directions):
        # returns True if a move in a certain direction is possible      
        for x in xrange(self.size):
            for y in xrange(self.size):
                # if current tile is occupied
                if self.data[x][y]:
                    # examine adjacent tile value
                    for i in directions:
                        move = vectors[i]
                        adjacent_value = self.get_tile_value((x + move[0], y + move[1]))
                        # True if value is the same or adjacent tile is empty
                        if adjacent_value == self.data[x][y] or adjacent_value == 0:
                            return True
                # else if current tile is empty
                elif self.data[x][y] == 0:
                    return True
        # else, cannot move in the specific direction
        return False
      
    def get_available_moves(self, directions = directions):
        # return a list of available moves at a certain state
        available_moves = []
        # check all four directions
        for direction in directions:
            grid_copy = self.clone() # move copies instead of original grid
            if grid_copy.move(direction):
                # if move method returns True, the direction is an available move
                available_moves.append(direction)
        return available_moves
      
    def get_tile_value(self, pos):
        # access the value of a tile
        x = pos[0]
        y = pos[1]
        # if x and y are not outside of the grid
        if not (x < 0 or x >= self.size or y < 0 or y >= self.size):
            # return value in tile
            return self.data[x][y]
        else:
            return None
          
    def display(self,grid):
        # display list of lists as a grid-looking representation with colors
        def color(x):
            if x == 0:    return Fore.WHITE + Back.WHITE
            if x == 2:    return Fore.RED + Back.WHITE
            if x == 4:    return Fore.GREEN + Back.WHITE
            if x == 8:    return Fore.BLACK + Back.WHITE
            if x == 16:   return Fore.BLUE + Back.WHITE
            if x == 32:   return Fore.MAGENTA + Back.WHITE
            if x == 64:   return Fore.CYAN + Back.WHITE
            if x == 128:  return Fore.WHITE + Back.CYAN
            if x == 256:  return Fore.WHITE + Back.MAGENTA
            if x == 512:  return Fore.WHITE + Back.BLUE
            if x == 1024: return Fore.WHITE + Back.RED
            if x == 2048: return Fore.WHITE + Back.GREEN
            if x == 4096: return Fore.WHITE + Back.BLACK
        for i in grid.data:
            for j in i:
                print color(j) + ("%4d" % j) + Fore.RESET + Back.RESET,
            print
      
    def user_play(self):
        # method to allow user to make moves on the grid and play 2048
        # not used in the solver implementation
        g = Grid()
        g.data[0][0] = 2
        g.data[1][0] = 2
        g.data[3][0] = 4
        while True:
            for i in g.data:
                print i
            print "Available moves:", g.get_available_moves()
            print "0 = Up, 1 = Down, 2 = Left, 3 = Right"
            g.move(raw_input("What is your move? "))

### **Game Manager Class**

In [0]:
from random import randint, random
from time import time

initial_tiles = 2       # number of tiles at the beginning of game
possibility_of_2 = 0.9  # possibility of spawning 2 compared to 4
action_dictionary = {0:"Up", 1:'Down', 2:'Left', 3:'Right'}

class Game:
    """Represents the Central Game Manager for the 2048-puzzle.
    
    Parameters
    ----------
    grid : Grid object
        Instantiate grid object of a specific size.
    possibility : int
        Represents the possibility of new spawns being 2 and 4
    init_tile : int
        Represents the number of tiles at the beginning of the game
    interactive : bool
        Whether or not to print the grid and direction at each game state
    depth_limit : int
        Represents the depth limit for depth-limited search
        
    """
    def __init__(self, size = n, interactive = False, depth_limit = 4):
        self.grid = Grid(size)
        self.possibility = possibility_of_2
        self.init_tiles = initial_tiles
        self.interactive = interactive
        self.depth = depth_limit
        
    def get_random_empty_tile(self, grid):
        # if at least one empty tile exists, pick one at random
        empty_tiles = grid.get_empty_tiles()
        if empty_tiles:
            return empty_tiles[randint(0, len(empty_tiles) - 1)] 
        else:
            return None
        
    def start(self, heur):
        # starts the game
        for i in xrange(self.init_tiles):
            # insert random initial tiles
            self.insert_initial_tile()
            
        # show the iniial grid state
        self.grid.display(self.grid)
        
        max_tile = 0  # value of the largest tile  
        start_time = time()
        moves = []    # record all the moves that were made
        # run while game is not over and 2048 is not achieved
        while not self.game_over() and not max_tile == 2048:
            grid = self.grid
            
            # search for a move that returns highest expected value
            move = expectimax(grid, True, self.depth, heur)[1]
            moves.append(move)
            
            if self.interactive:
                print "\nMaking move: {}".format(action_dictionary[move])
            
            # validate move to be an integer [0,3] and apply the move
            if move != None and move >= 0 and move < 4:
                if self.grid.can_move([move]):
                    self.grid.move(move)
                    max_tile = self.grid.get_largest_tile()
                else:
                    print "Grid cannot move in this direction"
            else:
                print "Invalid move"
               
            # once a move is made, 2 or 4 is spawned randomly at an empty tile     
            empty_tile = self.get_random_empty_tile(grid)
            new_tile_value = self.new_tile_value()
            if empty_tile:
                self.grid.set_value(empty_tile, new_tile_value)           
            if self.interactive:
                self.grid.display(self.grid)                
        end_time = time()    
        if max_tile == 2048:
            if not self.interactive:
                print "\nCompleted grid state"
                self.grid.display(self.grid)
            print "\nGoal achieved!"
            print "Time Taken: %.2fs" % (end_time - start_time)
            print "Moves per second: %d" % int(len(moves) / (end_time - start_time))
            print "Proportion of up moves: %.2f" % (float(moves.count(0)) / len(moves))
            print "Proportion of down moves: %.2f" % (float(moves.count(1)) / len(moves))
            print "Proportion of left moves: %.2f" % (float(moves.count(2)) / len(moves))
            print "Proportion of right moves: %.2f" % (float(moves.count(3)) / len(moves))
        else:
            print "\nGame over!"
            print "Largest tile achieved was {}".format(max_tile)
        
    def game_over(self):
        # game over when grid can no longer move
        return not self.grid.can_move()
      
    def new_tile_value(self):
        # spawn 2 at a given possibility (default is 0.9)
        if random() < possibility_of_2:
            return 2
        # spawn 4 otherwise
        else:
            return 4
          
    def insert_initial_tile(self):
        # insert 2 or 4 in initial tiles
        tile_value = self.new_tile_value()
        tiles = self.grid.get_empty_tiles()
        tile = tiles[randint(0, len(tiles) - 1)]
        self.grid.set_value(tile, tile_value)

### **Expectimax Algorithm and Heuristics**

In [0]:
from math import log, fabs
import itertools
import numpy as np

def expectimax(grid, max_step, depth, heur):
    '''
    Tree search algorithm that maximizes expected utilities.
    Depth-limited search with depth specified by user.
    '''
    empty_tiles = grid.get_empty_tiles()
    stop_threshold = 0     # stopping condition

    # if depth-limit exceeded, stop recursion and call heuristic function
    if depth <= stop_threshold:
        return [heur(grid), None]

    expected_value = None  # expected value of a subtree
    best_move = None       # move that maximizes expected value
    
    # find a move that maximizes expected_value
    if max_step:
        moved = None
        for direction in directions:
            # move a copy of the grid in each direction
            grid_copy = grid.clone()
            grid_copy.move(direction)
            # direction does not change grid, try another direction
            if grid.equal(grid_copy):
                continue
            # if direction is valid, call e-step and recurse into sub-tree
            moved = True
            e_step = expectimax(grid_copy, not max_step, depth-1, heur)[0]
            
            # record maximum expected value and its corresponding move
            if expected_value == None or e_step > expected_value:
                expected_value = e_step
                best_move = direction
                
        # no possible moves
        if not moved:
            return heur(grid), None
          
    # e-step: calculate expected value of a sub-tree up to the depth limit
    else:
        # array that stores all the possible next states 
        moves = [[x, y, 2] for x, y in empty_tiles] + [[x, y, 4] for x, y in empty_tiles]
        total_moves = float(len(moves))
        sub_tree_payoff = 0  # expected utility of a decision
        # calculate expectation over all possible tile spawns
        for x, y, val in moves:
            grid_copy = grid.clone()
            # temporarily set 2 or 4 in an empty tile
            grid_copy.set_value((x, y), val)
            # sum up the expected values at each state of a sub-tree
            sub_tree_payoff += expectimax(grid_copy, not max_step, depth-1, heur)[0] / total_moves
            # reset tile back to empty
            grid_copy.set_value((x, y), 0) 
        expected_value = sub_tree_payoff

    return expected_value, best_move

In [0]:
def heur_monotonic(grid):
    # build monotonically decreasing tiles near the corner piece to ease merging
    grid_list = list(itertools.chain(*grid.data))
    empty_tiles = len([i for i, x in enumerate(grid_list) if x == 0])
    score = 0
    weights = [2 ** i for i in range(len(grid_list),0,-1)]
    for i in xrange(len(grid_list)):
        # for all values in grid greater or equal than 8
        if grid_list[i] >= 8:
            # monotonically decreasing values on top row from left to right
            # to ease merging with max_tile in top left corner
            score += weights[i] * (log(grid_list[i]) / log(2))
    return score / (len(grid_list) - empty_tiles)
        
def heur_smoothness(grid):
    # minimize differences between neighboring tiles to encourage merging
    # uses logarithmic distance with base 2
    score = 0
    for i in range(grid.size):
        for j in range(grid.size):
            if grid.data[i][j] != 0:
                # if tile is not 0, store its value power
                value = log(grid.data[i][j]) / log(2)
                # iterate to the rest of columns to the right
                # penalize the score if their logarithmic distance is large
                for k in range(grid.size - 1 - j):
                    next_right = grid.data[i][j + k + 1]
                    if next_right != 0:
                        right_value = log(next_right)/log(2)
                        if right_value != value:
                            score -= fabs(right_value - value)
                            break
                # repeat with tiles below 
                for k in range(grid.size - 1 - i):
                    next_down = grid.data[i + k + 1][j]
                    if next_down != 0:
                        down_value = log(next_down) / log(2)
                        if down_value != value:
                            score -= fabs(down_value - value)
                            break
    return score 
  
def heur_empty_tiles(grid):
    # heuristic to maximize number of empty tiles on the grid
    list_of_empty_tiles = grid.get_empty_tiles()
    return len(list_of_empty_tiles)
  
def heur_corner(grid):
    # place highest value tile in the top left corner 
    # flatten list to improve performance
    grid_list = list(itertools.chain(*grid.data))
    empty_tiles = len([i for i, x in enumerate(grid_list) if x == 0])
    max_tile = max(grid_list)  # value of max tile
    score = 0
    weights = [2 ** i for i in range(len(grid_list),1,-1)] # [65536, 32768,..., 2]
    # if max tile is in the top left corner
    if max_tile == grid_list[0]:
        # sum its power multiplied by the largest possible weight
        # bonus for keeping the tile in the top left corner
        score += (log(grid_list[0]) / log(2)) * weights[0]
    return score/(len(grid_list) - empty_tiles), max_tile
  
def heuristic(grid):
    # adds all the heuristic scores
    return 0.1 * heur_smoothness(grid) + log(heur_corner(grid)[1]) / log(2) + \
  heur_empty_tiles(grid) + heur_corner(grid)[0] + heur_monotonic(grid)

## Execution

In [21]:
def main():
    print "Starting AI solver for the 2048-Puzzle... \n"
    print "Initial grid state:"
    game = Game(interactive = False, depth_limit = 3)
    game.start(heur = heuristic)
    
if __name__ == '__main__':
    main()  

Starting AI solver for the 2048-Puzzle... 

Initial grid state:
   0    0    0    0
   0    0    0    0
   0    0    2    0
   2    0    0    0

Completed grid state
2048  128    0    2
  32   64    8    2
   4    8    2    0
   8    2    0    0

Goal achieved!
Time Taken: 19.09s
Moves per second: 54
Proportion of up moves: 0.44
Proportion of down moves: 0.01
Proportion of left moves: 0.35
Proportion of right moves: 0.21


## **Future Improvements to the Solver**

There are multiple aspects of this implementation that could be improved that are performance related and not-performance related.

### Performance-Related

As stated above, the speed-accuracy tradeoff of this implementation is large, even between depth limits 3 and 4. A depth limit of 4 is 10 times slower than depth limit of 3, but 1.5x more likely to complete a 2048 tile. Therefore, there is a need for speed-related improvements that can be made to maintain high accuracy and relatively low runtimes.

1.   Optimizing data structures - implement a highly-optimized data structure such as using bit values to represent integers and a table to represent the grid and heuristic scores. [This GitHub implementation](https://github.com/nneonneo/2048-ai) uses such a methodology and also use table lookups for heuristic scores and the representation of grids after each move. This improvement would help trim the solver runtime by allowing the AI to search a huge number of game states in a short period of time.
2.   Dynamic programming - a large contributor to slow runtimes is the recursion of the expectimax algorithm. One can prevent the algorithm from recursing into states it had previously encountered using a [transposition table](https://en.wikipedia.org/wiki/Transposition_table), which is essentially memoization applied to the expectimax tree search. The search should terminate and yield the result much faster than having the explore all the nodes. This could be equivalent to pruning the tree and reducing state spaces.

Further improving this implementation's accuracy is also possible.
1.    Optimally weight heuristic scores - currenly, I have a large weight on the corner heuristic while small weight on the smoothness heuristic which is based on my style of play. To find the optimal distribution of weights for each heuristic, we need to use multivariable optimization perhaps by running the algorithm over and over again while making incremental adjustments (i.e. hill-climbing) or incorporate stochasic processes to reduce likelihood or arriving at local optima rather than the global optima (i.e. simulated annealing). 
2.    Reinforcement learning - this method would use a non-heuristic approach to learn the game, but use its historic gameplay data to figure out the best moves to make at each state. This approach would improve over time, as the AI plays more and more games. I have no doubt that a sufficiently well-trained AI will beat my implementation in speed and accuracy, since data is stored and processed to refine the AI over time.


### Not Performance-Related

1.    Improving color schemes of this implementation - since the colors from the colorama are limited, there are some colors used in this implementation that are not as easy to the eye as they could be. An improvement to this is to use [ANSI escape sequences](http://jafrog.com/2013/11/23/colors-in-terminal.html) which can display up to 16 million colors (though it has to be run on a terminal, instead of being within a notebook)
2.    Integrate this AI solver with the actual [2048 game interface](https://gabrielecirulli.github.io/2048/) - it would be cool to be able to watch this AI solve the game on the original platform, though this may require screen capture to "see" the tiles and port the output of our algorithm as a keyboard button press. However, this is out of the scope of this project for now.

# References

1. Bednova, I. (2013). *Colors In Terminal. JAFROG'S DEV BLOG.* Retrieved from http://jafrog.com/2013/11/23/colors-in-terminal.html
2. Giraud, F. (2014). *Simple 2048 game implementation in Python. GitHub Repository.* Retrieved from https://github.com/fede1024/2048
3. Klein, D., Abbeel, P. (2018). *Lecture 7: Expectimax Search and Utilities. UC Berkeley CS188 Intro to AI -- Course Materials* Retrieved from http://ai.berkeley.edu./lecture_slides.html
4. Shah, S. (2016). *Minimax algorithm for 2048 game. GitHub Repository.* Retrieved from https://github.com/ss4936/2048
5. Shoaran, M. (2009). *Expectimax Search. University of Victoria.* Retrieved from https://web.uvic.ca/~maryam/AISpring94/Slides/06_ExpectimaxSearch.pdf
6. Xiao, R. (2014). *What is the optimal algorithm for the game 2048?. Stack Overflow.* Retrieved from https://stackoverflow.com/a/22498940
7. Xiao, R. (2017). *AI for the 2048 game. GitHub Repository.* Retrieved from https://github.com/nneonneo/2048-ai
8. Zettlemoyer, L. (2011). *Adversarial Search. University of Washington Computer Science & Engineering.* Retrieved from https://courses.cs.washington.edu/courses/cse473/11au/slides/cse473au11-adversarial-search.pdf